In [299]:
# Importando as bibliotecas necessárias 
import mysql.connector
from mysql.connector import Error
import pandas as pd 
import csv 
import re
from google.oauth2 import service_account
from googleapiclient.discovery import build


In [300]:
# Configuração da conexão
config = {
    'user': 'root',
    'password': 'Pingofofo01',
    'host': 'localhost',
    'database': 'otica_mais_visao',
    'raise_on_warnings': True
}
# Iniciando a conexão
try:
    conn = mysql.connector.connect(**config)
    print('Conexão bem-sucedida!')

except mysql.connector.Error as err:
    print(f'Erro ao conectar ao banco de dados: {err}')
finally:
    # Fechando a conexão
    if conn.is_connected():
        conn.close()
        print('Conexão fechada.')

Conexão bem-sucedida!
Conexão fechada.


In [301]:
try:
    # Estabelecendo a conexão
    conn = mysql.connector.connect(**config)
    delete_tabelas = conn.cursor()
    # Desativando restrições de chave estrangeira
    delete_tabelas.execute("SET FOREIGN_KEY_CHECKS = 0")

    # Obter a lista de tabelas
    delete_tabelas.execute("SHOW TABLES")
    tables = delete_tabelas.fetchall()

    for table in tables: 
        table_name = table[0]
        drop_table_query = f"DROP TABLE {table_name}"
        delete_tabelas.execute(drop_table_query)
        print(f"Foi excluido com sucesso: {table_name}")

    # Ativando restrições de chave estrangeira novamente
    delete_tabelas.execute("SET FOREIGN_KEY_CHECKS = 1")

    conn.commit()

except mysql.connector.Error as  err:
        print(f"Erro ao excluir registros: {err}")

finally: 
        if delete_tabelas:
            delete_tabelas.close()
        if  conn.is_connected():
             conn.close()
             print("Conexão fechada.")


Foi excluido com sucesso: armacao
Foi excluido com sucesso: clientes
Foi excluido com sucesso: estoque
Foi excluido com sucesso: fluxo_caixa
Foi excluido com sucesso: fornecedores
Foi excluido com sucesso: funcionarios
Foi excluido com sucesso: lente
Foi excluido com sucesso: mercadoria
Foi excluido com sucesso: transacao
Foi excluido com sucesso: vendas
Conexão fechada.


In [302]:
# Criando a tabela
def criar_tabela():
    try:
        conn = mysql.connector.connect(**config)
        cursor = conn.cursor()

        # Consultas SQL para criar a tabela
        CREATE_TABLE_ARMACAO = """
            create table armacao (
            id_armacao varchar(5) not null primary key, 
            modelo varchar(255),
            marca varchar(255), 
            tamanho varchar(255) 
        );     
        """
        CREATE_TABLE_LENTE = """
            create table lente ( 
            id_lente varchar(5) not null primary key, 
            marca varchar(255),
            tamanho varchar(255),
            tipo varchar(255) 
        );
        """
        CREATE_TABLE_ESTOQUE = """
            create table estoque ( 
            codigo_barras varchar(12) not null primary key, 
            quantidade int(10), 
            id_fornecedor varchar(10),
            descricao varchar(1000), 
            preco_unitario decimal(10,2) 
        );
        """
        CREATE_TABLE_MERCADORIA = """
            CREATE TABLE mercadoria (
            id_produto varchar(5) NOT NULL PRIMARY KEY,
            nome_produto varchar(255),
            descricao_produto varchar(1000),
            codigo_barras_produto varchar(12),
            valor_unitario_produto decimal(10,2),
            tipo_produto varchar(255), 
            estoque int(11)
        );
        """ 

        CREATE_TABLE_FORNECEDORES = """
            CREATE TABLE fornecedores (
            id_fornecedor varchar(10) NOT NULL PRIMARY KEY,
            telefone VARCHAR(11),
            razao_social VARCHAR(255),
            cnpj VARCHAR(14),
            endereco VARCHAR(255)
        );
        """
        CREATE_TABLE_CLIENTE = """
            create table clientes ( 
            cpf varchar(11) not null primary key, 
            telefone varchar(11),
            nome varchar(255), 
            endereco varchar(255) 
        ); 
        """
        CREATE_TABLE_FUNCIONARIOS = """
            create table funcionarios ( 
            id_vendedor varchar(4) not null primary key, 
            nome varchar(255),
            telefone varchar(11), 
            endereco varchar(255), 
            cpf varchar(11),
            valor_vendido decimal(15,2) 
        ); 
        """ 
        CREATE_TABLE_TRANSACAO = """
            CREATE TABLE transacao (
            id_transacao varchar(10) NOT NULL PRIMARY KEY,
            id_produto varchar(5) UNIQUE KEY,
            valor_unitario DECIMAL(10,2),
            quantidade INT(10),
            data_venda varchar(14),
            valor_total DECIMAL(10,2)
        );
        """
        CREATE_TABLE_FLUXO_CAIXA = """
            create table fluxo_caixa ( 
            data_venda varchar(14) NOT NULL PRIMARY KEY,
            despesas decimal(10,2), 
            descricao_despesas varchar(255),
            entradas decimal(10,2), 
            saldo_operacional decimal(10,2) 
        ); 
        """
        CREATE_TABLE_VENDAS = """
            CREATE TABLE vendas (
            id_venda varchar(7) NOT NULL PRIMARY KEY,
            valor_total DECIMAL(10,2),  
            cpf_cliente varchar(11) not null,  
            data_venda varchar(14),
            id_vendedor varchar(4),
            id_transacao varchar(10)
        );
        """   
    

     # Executando a consulta SQL
        cursor.execute(CREATE_TABLE_ARMACAO)
        cursor.execute(CREATE_TABLE_LENTE)
        cursor.execute(CREATE_TABLE_ESTOQUE)
        cursor.execute(CREATE_TABLE_MERCADORIA)
        cursor.execute(CREATE_TABLE_FORNECEDORES)
        cursor.execute(CREATE_TABLE_CLIENTE)
        cursor.execute(CREATE_TABLE_FUNCIONARIOS)
        cursor.execute(CREATE_TABLE_TRANSACAO)
        cursor.execute(CREATE_TABLE_FLUXO_CAIXA)
        cursor.execute(CREATE_TABLE_VENDAS)
        print("Tabelas criadas com sucesso")

    except mysql.connector.Error as err:
        print(f'Erro ao criar as tabelas: {err}')
        
    finally:
        # Fechando a conexão
        if 'conn' in locals() and conn.is_connected():
            cursor.close()
            conn.close()
            print('Conexão fechada.')

criar_tabela()


Tabelas criadas com sucesso
Conexão fechada.


In [303]:
# Inserindo dados nas tabelas através de arquivos csv
try:
   conn = mysql.connector.connect(**config)
   cursor = conn.cursor()

   with open('clientes.csv', 'r') as clientes_csv:
      leitor_csv = csv.reader(clientes_csv, delimiter=';')
      dados = list(leitor_csv)

   insert_clientes = """
   INSERT INTO clientes (
      cpf
   ,  telefone
   ,  nome
   ,  endereco
   ) 
   VALUES (%s, %s, %s, %s)
   """
   # Executar a inserção das linhas
   print(dados)
   cursor.executemany(insert_clientes, dados)
   # Confirmar as alterações
   conn.commit()
   # Fechar a conexão
   conn.close()

except mysql.connector.Error as err:
        print(f'Erro ao alimentar tabelas: {err}')
        
finally:
        # Fechando a conexão
        if 'conn' in locals() and conn.is_connected():
            cursor.close()
            conn.close()
            print('Conexão fechada.')


[['12345678911', '81991102340', 'Maria Alice', 'Rua Antonio Falcão'], ['24397810288', '79908235670', 'Ana Geovanna', 'Conselheiro Rosa e Silva'], ['72898717003', '71995678932', 'Luciana Batista', 'Rua dos Carijós'], ['25798052509', '61987603425', 'Igor Vilarim', 'Rua Domingos Olímpio'], ['90762891547', '85976345098', 'Juliana Siqueira', 'Rua Arlindo Nogueira'], ['22794093401', '91990507891', 'Lucas Gomes', 'Rua Carlos Augusto Cornelsen'], ['44806523917', '85998702389', 'Pedro Gustavo', 'Avenida Rio Branco'], ['17537840948', '81996789023', 'Edmilson Correia', 'Rodovia Raposo Tavares'], ['44294133812', '11997803450', 'Leticia Nunes', 'Avenida Governador José Malcher'], ['40120248020', '11998709870', 'Luisa Silva', 'Avenida Desembargador Moreira'], ['20430811145', '21998034787', 'Cassio Alves', 'Travessa da CDL'], ['11348170132', '71993704560', 'Victor Costa', 'Rua Frederico Moura'], ['20974822271', '51994502389', 'Maria Bravo', 'Rua Tenente-Coronel Cardoso'], ['40908539205', '51998304020

In [304]:
# Inserindo dados nas tabelas através de arquivos csv
try:
   conn = mysql.connector.connect(**config)
   cursor = conn.cursor()

   with open('lentes.csv', 'r') as lentes_csv:
      leitor_csv = csv.reader(lentes_csv, delimiter=';')
      dados = list(leitor_csv)

   insert_lentes = """
   INSERT INTO lente (
      id_lente
   ,  marca
   ,  tamanho
   ,  tipo
   ) 
   VALUES (%s, %s, %s, %s)
   """
   # Executar a inserção das linhas
   print(dados)
   cursor.executemany(insert_lentes, dados)
   # Confirmar as alterações
   conn.commit()
   # Fechar a conexão
   conn.close()

except mysql.connector.Error as err:
        print(f'Erro ao alimentar tabelas: {err}')
        
finally:
        # Fechando a conexão
        if 'conn' in locals() and conn.is_connected():
            cursor.close()
            conn.close()
            print('Conexão fechada.')

[['95550', ' HOYA', '30.25mm', 'Antirreflexo blue control'], ['45480', ' HOYA', '66.21mm', 'Antirreflexo Long life'], ['47749', ' HOYA', '42.46mm', 'Antirreflexo NO-risk'], ['94725', ' HOYA', '05.60mm', 'Antirreflexo CleanExtra'], ['68775', ' HOYA', '95.63mm', 'Antirreflexo Aqua'], ['65616', 'Carl Zeiss', '86.18mm', 'Antireflexo a zeiss'], ['42639', 'Carl Zeiss', '43.40mm', 'DuraVision Platinum'], ['51376', 'Carl Zeiss', '85.81mm', 'DuraVision Blue Protector'], ['98748', 'Carl Zeiss', '02.13mm', 'DuraVision Chrome'], ['63601', 'Carl Zeiss', '71.53mm', 'DuraVision silver'], ['44411', 'Carl Zeiss', '39.63mm', 'Zeiss DriveSafe'], ['18547', 'Lentes Essilor(Crisal e Varilux)', '38.13mm', ' lentes Crizal'], ['53618', 'Lentes Essilor(Crisal e Varilux)', '85.96mm', ' lentes Crizal prevencia'], ['69159', 'Lentes Essilor(Crisal e Varilux)', '40.21mm', ' lentes Crizal Sapphire HR'], ['33406', 'Lentes Essilor(Crisal e Varilux)', '50.70mm', ' lentes Crizal Rock'], ['10809', 'Lentes Essilor(Crisal e

In [305]:
# Inserindo dados nas tabelas através de arquivos csv
try:
   conn = mysql.connector.connect(**config)
   cursor = conn.cursor()

   with open('armacao.csv', 'r') as armacao_csv:
      leitor_csv = csv.reader(armacao_csv, delimiter=';')
      dados = list(leitor_csv)

   insert_armacao = """
      INSERT INTO armacao (
      id_armacao
   ,  modelo
   ,  marca
   ,  tamanho
   ) 
   VALUES (%s, %s, %s, %s)
   """
   # Executar a inserção das linhas
   print(dados)
   cursor.executemany(insert_armacao, dados)
   # Confirmar as alterações
   conn.commit()
   # Fechar a conexão
   conn.close()

except mysql.connector.Error as err:
        print(f'Erro ao alimentar tabelas: {err}')
        
finally:
        # Fechando a conexão
        if 'conn' in locals() and conn.is_connected():
            cursor.close()
            conn.close()
            print('Conexão fechada.')

[['11111', 'KP1112', 'KIPLING', '51'], ['22222', 'VO5043L', 'VOGUE', '54'], ['33333', 'CKJ18703', 'CK JEANS', '53'], ['44444', 'SK5452', 'SWAROVSKI', '52'], ['55555', 'AT7072', 'ATITUDE', '50'], ['66666', 'OOX8156', 'OAKLEY', '56'], ['77777', 'DK5000', 'DKNY', '51'], ['88888', '7130', 'NIKE', '54'], ['99999', 'BG7073I', 'BULGET', '52'], ['12121', 'CKJ20636', 'CK JEANS', '52'], ['13131', 'AT8093', 'ATITUDE', '60'], ['14141', 'RB3519L', 'RAYBAN', '63'], ['15151', 'L986', 'LACOSTE', '52'], ['16161', 'AN4251', 'ARNETTE', '58'], ['17171', 'DS68', 'EVOKE', '57']]


In [306]:
# Inserindo dados na tabela de fornecedores 
try:
    conn = mysql.connector.connect(**config)
    cursor = conn.cursor()
    
    with open ('fornecedores.csv', 'r') as fornecedores_csv:
        leitor_csv = csv.reader(fornecedores_csv, delimiter=';')
        dados = list(leitor_csv)

    insert_fornecedores = """
        INSERT INTO fornecedores (
              id_fornecedor
            , telefone
            , razao_social
            , cnpj
            , endereco 
        )
        VALUES (%s, %s, %s, %s, %s)
    """
    # Executar a inserção das linhas
    print(dados)
    cursor.executemany(insert_fornecedores, dados)
    # Confirmar as alterações
    conn.commit()
    # Fechar a conexão
    conn.close()
    
except mysql.connector.Error as err:
        print(f'Erro ao alimentar tabelas: {err}')
finally:
     #Fechar a conexão
    if 'conn' in locals() and conn.is_connected():
        cursor.close()
        conn.close()
        print('Conexão fechada.')

[['1590713659', '64933692843', 'salvanguarda otica', '71221800000000', '  Rua Avenida Tocantins   bairro Vila Jardim Rio Claro   cidade Jataí   estado GO'], ['2147483647', '69932926887', 'santo otica', '38886400000000', '   Rua Travessa da CDL   bairro Centro   cidadeJi Paraná   estado RO'], ['6665644440', '28932098883', 'wire otica', '85156900000000', '   Rua Avenida Esbertalina Barbosa Damiani   bairro Guriri Norte   cidade São Mateus   estado ES'], ['7651368520', '19929374414', 'otica labs', '12765700000000', '   Rua Barão de Vitória   bairro Casa Grande   cidade Diadema   estado SP'], ['7093670701', '17933135389', 'analytic otica', '21725300000000', '   Rua Cristiano Olsen   bairro Jardim Sumaré   cidade Araçatuba   estado SP'], ['3411554087', '45927296415', 'otica sacan', '73715200000000', ' Rua Carlos Augusto Cornelsen   bairro Bom Retiro   cidade Curitiba   estado PR'], ['4032103692', '11933815776', 'son otica', '37251800000000', '  Rua Praça da República   bairro República   ci

In [307]:
#Inserindo dados na tabela de mercadorias
try:
    conn = mysql.connector.connect(**config)
    cursor = conn.cursor()

    with open ('mercadoria.csv', 'r') as mercadorias_csv:
        leitor_csv = csv.reader(mercadorias_csv, delimiter=';')
        dados = list(leitor_csv)

    insert_mercadorias = """
        INSERT INTO mercadoria (   
            id_produto,
            nome_produto,
            descricao_produto,
            codigo_barras_produto,
            valor_unitario_produto,
            tipo_produto, 
            estoque
            )
            VALUES (%s, %s, %s, %s, %s, %s, %s)
            """
    # Executar a inserção das linhas
    print(dados)
    cursor.executemany(insert_mercadorias, dados)
    # Confirmar as alterações
    conn.commit()
    # Fechar a conexão
    conn.close()

except mysql.connector.Error as err:
        print(f'Erro ao alimentar tabelas: {err}')
finally:
        #Fechar a conexão
        if 'conn' in locals() and conn.is_connected():
            cursor.close()
            conn.close()
            print('Conexão fechada.')

[['95550', 'Antirreflexo blue control', 'redução de reflexos e filtros de luz azul nociva', '125439872163', '156.9', 'Lente', '30'], ['45480', 'Antirreflexo Long life', 'Durabilidade excepcional', '987654321012', '156.9', 'Lente', '12'], ['47749', 'Antirreflexo NO-risk', 'Sem riscos e reflexos indesejados', '543210987654', '274.37', 'Lente', '23'], ['94725', 'Antirreflexo CleanExtra', 'Limpeza fácil e visão clara', '789654321012', '231.85', 'Lente', '21'], ['68775', 'Antirreflexo Aqua', 'Proteção contra água e sujeira', '210987654321', '236.18', 'Lente', '22'], ['65616', 'Antireflexo a zeiss', 'Qualidade premium da Zeiss', '456789012345', '269.87', 'Lente', '26'], ['42639', 'DuraVision Platinum', 'Resistência máxima', '876543210987', '650.06', 'Lente', '2'], ['51376', 'DuraVision Blue Protector', 'Proteção contra luz azul', '123456789012', '500.5', 'Lente', '13'], ['98748', 'DuraVision Chrome', 'Estilo e durabilidade', '987654321098', '550.75', 'Lente', '24'], ['63601', 'DuraVision sil

In [308]:
#Inserindo dados na tabela de estoque 

try: 
    conn = mysql.connector.connect(**config)
    cursor = conn.cursor()

    with open ('estoque.csv', 'r') as estoque_csv:
        leitor_csv = csv.reader(estoque_csv, delimiter=';')
        dados = list(leitor_csv)

    insert_estoque = """
        INSERT INTO estoque (
            codigo_barras, 
            quantidade, 
            id_fornecedor,
            descricao, 
            preco_unitario
            )
            VALUES (%s, %s, %s, %s, %s)
            """
    # Executar a inserção das linhas
    print(dados)
    cursor.executemany(insert_estoque, dados)
    # Confirmar as alterações
    conn.commit()
    # Fechar a conexão
    conn.close()
except mysql.connector.Error as err:
        print(f'Erro ao alimentar tabelas: {err}')
finally:
        #Fechar a conexão
        if 'conn' in locals() and conn.is_connected():
            cursor.close()
            conn.close()
            print('Conexão fechada.')

[['768239054788', '30', '1590713659', 'Lente', '156.9'], ['328417950621', '12', '2147483647', 'Lente', '249.99'], ['564032189503', '23', '6665644440', 'Lente', '274.37'], ['789250647103', '21', '7651368520', 'Lente', '231.85'], ['215903478906', '22', '7093670701', 'Lente', '236.18'], ['936857102493', '26', '3411554087', 'Lente', '269.87'], ['681045932718', '2', '4032103692', 'Lente', '650.06'], ['395874260713', '13', '3468956978', 'Lente', '500.5'], ['527143906841', '24', '9987298030', 'Lente', '550.75'], ['817563024935', '27', '5829454379', 'Lente', '600.25'], ['406382157904', '22', '8926870038', 'Lente', '650.8'], ['902745816301', '22', '8232769700', 'Lente', '400.9'], ['625409837164', '15', '4545116740', 'Lente', '450.6'], ['139578402516', '12', '4602339704', 'Lente', '550.4'], ['874305192683', '25', '7569652535', 'Lente', '600.1'], ['309764815207', '9', '1590713659', 'Lente', '500.2'], ['582034179506', '16', '2147483647', 'Lente', '650.3'], ['947361205384', '3', '6665644440', 'Lent

In [309]:
#Inserindo dados na tabela de funcionários
try:
    conn = mysql.connector.connect(**config)
    cursor = conn.cursor()

    with open ('funcionarios.csv', 'r') as funcionarios_csv:
        leitor_csv = csv.reader(funcionarios_csv, delimiter=';')
        dados = list(leitor_csv)

    insert_funcionarios = """
    INSERT INTO funcionarios (
            id_vendedor
            ,nome 
            ,telefone
            ,endereco 
            ,cpf 
            ,valor_vendido 
            )
        VALUES (%s, %s, %s, %s, %s, %s)
    """

    # Executar a inserção das linhas
    print(dados)
    cursor.executemany(insert_funcionarios, dados)
    # Confirmar as alterações
    conn.commit()
    # Fechar a conexão
    conn.close()
    
except mysql.connector.Error as err: 
    print(f'Erro ao alimentar tabelas: {err}')
finally:
    #Fechar a conexão
    if 'conn' in locals() and conn.is_connected():
        cursor.close()
        conn.close()
        print('Conexão fechada.')        


[['1234', 'Sophia Santos', '81985263652', 'Rua das Flores, 123, Bairro Jardim Primavera', '98654321987', '9015.5'], ['2345', 'Lucas Costa', '81945896322', 'Avenida Principal, 456, Bairro Centro', '12345678901', '9834.95'], ['5632', 'Isabella Silva', '81974588547', 'Travessa das Oliveiras, 789, Bairro Vila Esperança', '55522233344', '3465.84']]


In [310]:
#Inserindo dados na tabela de transações

try:
    conn = mysql.connector.connect(**config)
    cursor = conn.cursor()
    
    with open ('transacao.csv', 'r') as transacao_csv:
        leitor_csv = csv.reader(transacao_csv, delimiter=';')
        dados = list(leitor_csv)

        insert_transacao = """
        INSERT INTO transacao (
            id_transacao,
            id_produto,
            valor_unitario,
            quantidade,
            data_venda,
            valor_total
            )
        VALUES (%s, %s, %s, %s, %s, %s)
        """

    # Executar a inserção das linhas
    print(dados)
    cursor.executemany(insert_transacao, dados)
    # Confirmar as alterações
    conn.commit()
    # Fechar a conexão
    conn.close()

except mysql.connector.Error as err: 
    print(f'Erro ao alimentar tabelas: {err}')
finally:
    #Fechar a conexão
    if 'conn' in locals() and conn.is_connected():
        cursor.close()
        conn.close()
        print('Conexão fechada.')    



[['8742091', '95550', ' 156.90', '1', '01/04/2023', ' 156.90'], ['5703685', '45480', '249.99', '6', '01/04/2023', ' 1499.94'], ['9214568', '47749', ' 274.37', '2', '02/04/2023', ' 548.74'], ['6487301', '94725', ' 231.85', '3', '02/04/2023', ' 695.55'], ['1895432', '68775', ' 236.18', '2', '03/04/2023', ' 472.36'], ['3067958', '65616', ' 269.87', '1', '03/04/2023', ' 269.87'], ['5829034', '42639', ' 650.06', '5', '04/04/2023', ' 3250.30'], ['7931046', '11111', ' 695.35', '10', '04/04/2023', ' 6953.50'], ['4156278', '22222', ' 296.30', '3', '05/04/2023', ' 888.90'], ['2650893', '33333', ' 951.67', '5', '05/04/2023', ' 4758.36'], ['9046132', '44444', ' 376.12', '2', '06/04/2023', ' 752.24'], ['1278945', '55555', ' 258.36', '1', '06/04/2023', ' 258.36'], ['5398274', '66666', ' 154.36', '1', '07/04/2023', ' 154.36'], ['6723459', '15151', ' 156.90', '1', '08/04/2023', ' 156.90'], ['8201643', '16161', ' 749.99', '2', '09/04/2023', ' 1499.98']]


In [311]:
#Inserindo dados na tabela de vendas
try:
    conn = mysql.connector.connect(**config)
    cursor = conn.cursor()
    
    with open ('vendas.csv', 'r') as vendas_csv:
        leitor_csv = csv.reader(vendas_csv, delimiter=';')
        dados = list(leitor_csv)
        
    insert_vendas = """
        INSERT INTO vendas (   
            id_venda
            ,valor_total  
            ,cpf_cliente  
            ,data_venda
            ,id_vendedor
            ,id_transacao
            ) 
            VALUES (%s, %s, %s, %s, %s, %s)
        """
    # Executar a inserção das linhas
    print(dados)
    cursor.executemany(insert_vendas, dados)
    # Confirmar as alterações
    conn.commit()
    # Fechar a conexão
    conn.close()
    
except mysql.connector.Error as err: 
    print(f'Erro ao alimentar tabelas: {err}')
finally:
    #Fechar a conexão
    if 'conn' in locals() and conn.is_connected():
        cursor.close()
        conn.close()
        print('Conexão fechada.')        

[['8742091', '156.9', '12345678911', '01/04/2023', '1234', 'a2B3c4D5e6'], ['5703685', '1499.99', '24397810288', '01/04/2023', '2345', 'F7g8H9i0J'], ['9214568', '548.74', '72898717003', '02/04/2023', '5632', 'k1L2m3N4o'], ['6487301', '695.55', '25798052509', '02/04/2023', '1234', 'P5q6R7s8T'], ['1895432', '472.36', '90762891547', '03/04/2023', '2345', '9uA0bC1d2E'], ['3067958', '269.87', '22794093401', '03/04/2023', '5632', 'f3G4h5I6j'], ['5829034', '3250.33', '44806523917', '04/04/2023', '1234', 'K7l8M9n0O'], ['7931046', '6953.45', '17537840948', '04/04/2023', '2345', 'p1Q2r3S4t'], ['4156278', '888.88', '44294133812', '05/04/2023', '5632', '5Uv6W7x8Y'], ['2650893', '4758.36', '40120248020', '05/04/2023', '1234', 'Z9A0bC1d2'], ['9046132', '752.25', '20430811145', '06/04/2023', '2345', 'e3F4g5H6i'], ['1278945', '258.36', '11348170132', '06/04/2023', '5632', 'J7k8L9m0N'], ['5398274', '154.36', '20974822271', '07/04/2023', '1234', 'o1P2q3R4s'], ['6723459', '156.9', '40908539205', '08/04/20

In [313]:
#Inserindo dados na tabela de fluxo de caixa
try:
    conn = mysql.connector.connect(**config)
    cursor = conn.cursor()

    with open ('fluxo_caixa.csv', 'r') as fluxo_caixa_csv:
        leitor_csv = csv.reader(fluxo_caixa_csv, delimiter=';')
        dados = list(leitor_csv)

    insert_fluxo_caixa = """
        INSERT INTO fluxo_caixa (
            data_venda,
            despesas,
            descricao_despesas,
            entradas,
            saldo_operacional 
            )
            VALUES (%s, %s, %s, %s, %s)
            """
    # Executar a inserção das linhas
    print(dados)
    cursor.executemany(insert_fluxo_caixa, dados)
    # Confirmar as alterações
    conn.commit()
    # Fechar a conexão
    conn.close()
except mysql.connector.Error as err:
        print(f'Erro ao alimentar tabelas: {err}')
finally:
        #Fechar a conexão
        if 'conn' in locals() and conn.is_connected():
            cursor.close()
            conn.close()
            print('Conexão fechada.')

[['01/04/2023', '1656.89', 'Pagamento de serviços de contabilidade', '872.34', '784.55'], ['02/04/2023', '1244.29', 'Marketing', '461.55', '1567.29'], ['03/04/2023', '742.23', 'Gastos com materiais de escritório ', '193.78', '2115.74'], ['04/04/2023', '10203.78', 'Aquisição de equipamentos de exame de visão', '650.92', '11668.6'], ['05/04/2023', '5647.24', 'Despesas com energia elétrica e água', '812.1', '16503.74'], ['06/04/2023', '1010.61', 'Custos de treinamento', '287.43', '17226.92'], ['07/04/2023', '154.36', 'Assessoria jurídica', '524.76', '16856.52'], ['08/04/2023', '156.9', 'Pagamento de aluguel', '986.21', '16027.21'], ['09/04/2023', '1499.99', 'Investimento em programas de fidelidade e recompensas', '375.89', '17151.31']]
